In [6]:
# 1. Instalação das bibliotecas necessárias
# O flag -U garante a atualização para a versão mais recente
!pip install -q -U google-genai gTTS ffmpeg-python

import os
import sys
from google.colab import drive
from google.colab import userdata

# Tenta importar a biblioteca recém-instalada
try:
    from google import genai
except ImportError:
    print("\n⚠️ AVISO DE SISTEMA:")
    print("As bibliotecas foram instaladas, mas o Colab precisa recarregar.")
    print("👉 Vá no menu superior: 'Runtime' (Ambiente de execução) > 'Restart session' (Reiniciar sessão).")
    print("👉 Depois, execute esta célula novamente.")
    sys.exit("Parando execução para reinício do runtime.")

print("✅ Bibliotecas instaladas e carregadas.")

# 2. Montar o Google Drive e Configurar Diretório
try:
    # force_remount ajuda a garantir que a conexão está ativa
    drive.mount('/content/drive', force_remount=True)

    # Define o caminho da pasta
    WORK_DIR = "/content/drive/MyDrive/GeminiAssistant"

    # Cria a pasta se não existir
    os.makedirs(WORK_DIR, exist_ok=True)
    print(f"📂 Diretório de trabalho pronto: {WORK_DIR}")

except Exception as e:
    print(f"\n❌ Erro ao configurar o Drive: {e}")
    print("Dica: Verifique se você autorizou o acesso no pop-up.")
    # Não paramos o código aqui, pois você pode querer rodar sem o Drive,
    # mas as funções de salvar arquivos falharão depois.

# 3. Configuração da API Key (Método Seguro - Secrets)
print("\n🔑 Configurando autenticação...")

try:
    API_KEY = userdata.get('GOOGLE_API_KEY')
except Exception:
    API_KEY = None

# Validação
if not API_KEY:
    print("\n❌ Erro Crítico: Chave não encontrada nos Segredos do Colab.")
    print("---------------------------------------------------------------")
    print("COMO RESOLVER:")
    print("1. No menu lateral esquerdo do Colab, clique no ícone de chave (🔑).")
    print("2. Clique em 'Add new secret'.")
    print("3. Nome: GOOGLE_API_KEY")
    print("4. Valor: Cole sua chave (começa com AIza...).")
    print("5. Ative o botão 'Notebook access' ao lado do segredo criado.")
    print("---------------------------------------------------------------")
    raise ValueError("API Key não configurada nos Segredos.")

# Configura o cliente
try:
    client = genai.Client(api_key=API_KEY)
    print("✅ API Key configurada e Cliente Gemini inicializado com sucesso!")
except Exception as e:
    print(f"❌ Erro ao inicializar o cliente Gemini: {e}")


✅ Bibliotecas instaladas e carregadas.
Mounted at /content/drive
📂 Diretório de trabalho pronto: /content/drive/MyDrive/GeminiAssistant

🔑 Configurando autenticação...
✅ API Key configurada e Cliente Gemini inicializado com sucesso!


In [7]:
# --- IMPORTAÇÕES NECESSÁRIAS ---
import os
import time
import sys
from base64 import b64decode
from gtts import gTTS
from IPython.display import display, Javascript, Audio
from google.colab import output

# Importação específica para criar o objeto de dados inline
from google.genai import types

# --- FUNÇÕES AUXILIARES (JAVASCRIPT PARA MICROFONE) ---
RECORD = """
const sleep  = time => new Promise(resolve => setTimeout(resolve, time))
const b2text = blob => new Promise(resolve => {
  const reader = new FileReader()
  reader.onloadend = e => resolve(e.srcElement.result)
  reader.readAsDataURL(blob)
})
var record = time => new Promise(async resolve => {
  stream = await navigator.mediaDevices.getUserMedia({ audio: true })
  recorder = new MediaRecorder(stream)
  chunks = []
  recorder.ondataavailable = e => chunks.push(e.data)
  recorder.start()
  await sleep(time)
  recorder.onstop = async () => {
    blob = new Blob(chunks)
    text = await b2text(blob)
    resolve(text)
  }
  recorder.stop()
})
"""

def record_audio_colab(filename, sec=10):
    """Grava áudio usando o microfone do navegador via JS."""
    display(Javascript(RECORD))
    print(f"🎤 PREPARE-SE! Gravando por {sec} segundos...")
    s = output.eval_js('record(%d)' % (sec * 1000))
    b = b64decode(s.split(',')[1])

    with open(filename, 'wb') as f:
        f.write(b)
    print(f"✅ Áudio capturado e salvo em: {filename}")
    return filename

# --- CLASSE DO ASSISTENTE ---
class ColabGeminiAssistant:
    def __init__(self, work_dir):
        self.work_dir = work_dir
        self.audio_input = os.path.join(work_dir, "input_audio.webm")
        self.audio_output = os.path.join(work_dir, "response_audio.mp3")

        # Usamos o modelo que sabemos que existe na sua conta
        self.model_name = "gemini-flash-latest"

    def process_audio(self, retry_count=0):
        print(f"🧠 Enviando áudio diretamente para o Gemini ({self.model_name})...")

        try:
            # 1. Ler o arquivo de áudio localmente em bytes
            with open(self.audio_input, "rb") as f:
                audio_bytes = f.read()

            # 2. Gerar resposta enviando o áudio INLINE (sem upload prévio)
            # Isso evita erros de processamento de arquivo no servidor
            response = client.models.generate_content(
                model=self.model_name,
                contents=[
                    types.Content(
                        parts=[
                            types.Part(text="Ouça este áudio. Responda de forma curta, direta e conversacional em português."),
                            types.Part(inline_data=types.Blob(
                                mime_type="audio/webm",
                                data=audio_bytes
                            ))
                        ]
                    )
                ]
            )

            print(f"🤖 Gemini: {response.text}")
            return response.text

        except Exception as e:
            error_msg = str(e)

            # Tratamento de erro de cota (429)
            if "429" in error_msg or "RESOURCE_EXHAUSTED" in error_msg:
                if retry_count < 2:
                    print("\n⏳ Cota momentânea atingida. Aguardando 30s...")
                    time.sleep(30)
                    print("\n🔄 Tentando novamente...")
                    return self.process_audio(retry_count=retry_count + 1)

            print(f"❌ Erro ao processar áudio: {e}")
            return None

    def speak(self, text):
        if not text: return

        try:
            # Gera o áudio com gTTS
            tts = gTTS(text=text, lang='pt')
            tts.save(self.audio_output)

            # Toca o áudio no navegador
            print("🔊 Reproduzindo resposta...")
            display(Audio(self.audio_output, autoplay=True))
        except Exception as e:
            print(f"❌ Erro ao gerar áudio de resposta: {e}")

    def run(self, duration=10):
        # 1. Gravar
        record_audio_colab(self.audio_input, sec=duration)

        # 2. Processar
        response_text = self.process_audio()

        # 3. Falar
        self.speak(response_text)

# --- EXECUÇÃO ---
WORK_DIR = "/content/drive/MyDrive/GeminiAssistant"
assistant = ColabGeminiAssistant(WORK_DIR)

# Executa por 15 segundos
assistant.run(duration=15)




<IPython.core.display.Javascript object>

🎤 PREPARE-SE! Gravando por 15 segundos...
✅ Áudio capturado e salvo em: /content/drive/MyDrive/GeminiAssistant/input_audio.webm
🧠 Enviando áudio diretamente para o Gemini (gemini-flash-latest)...
🤖 Gemini: Parece que alguém tá com muita fome! Uma merenda caprichada agora cairia bem, né?
🔊 Reproduzindo resposta...
